<a href="https://colab.research.google.com/github/amodsgit/AmodTheCoder/blob/main/T5based_recipe_model_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Step 1: Install necessary packages
!pip -q install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.9 MB/s eta 0:00:00


In [3]:
# Step 2: Import necessary libraries
from transformers import FlaxAutoModelForSeq2SeqLM, AutoTokenizer
import requests
import json


In [4]:
# Step 3: Define the model and tokenizer
MODEL_NAME_OR_PATH = "flax-community/t5-recipe-generation"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH, use_fast=True)
model = FlaxAutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME_OR_PATH)

In [5]:
# Step 4: Define prefix and generation parameters
prefix = "items: "
generation_kwargs = {
    "max_length": 512,
    "min_length": 64,
    "no_repeat_ngram_size": 3,
    "do_sample": True,
    "top_k": 60,
    "top_p": 0.95
}

In [6]:
# Step 5: Define special tokens and their mapping
special_tokens = tokenizer.all_special_tokens
tokens_map = {
    "<sep>": "--",
    "<section>": "\n"
}

In [7]:
# Step 6: Define the function to skip special tokens
def skip_special_tokens(text, special_tokens):
    for token in special_tokens:
        text = text.replace(token, "")
    return text

In [8]:
# Step 7: Define the function for postprocessing the output
def target_postprocessing(texts, special_tokens):
    if not isinstance(texts, list):
        texts = [texts]

    new_texts = []
    for text in texts:
        text = skip_special_tokens(text, special_tokens)

        for k, v in tokens_map.items():
            text = text.replace(k, v)

        new_texts.append(text)

    return new_texts

In [9]:
COMMON_INGREDIENTS = ['salt','pepper']  # Define common ingredients here
API_NINJAS_KEY = "ElNmsjpzj8EURGIKbAj90Q==4iG8q4Fd43uSDQEP"  # Replace with your api-ninjas key

def estimate_calories(ingredients):
    total_calories = 0
    for ingredient in ingredients:
        # Request nutritional information from the food database
        response = requests.get('https://api.api-ninjas.com/v1/nutrition?query={}'.format(ingredient),
                                headers={'X-Api-Key': API_NINJAS_KEY})
        if response.status_code == requests.codes.ok:
            data = json.loads(response.text)
            if data:
                for item in data:
                    calories = item['calories']
                    print(f"Calories in {ingredient}: {calories}")
                    total_calories += calories
            else:
                print(f"Couldn't find nutritional information for {ingredient}")
        else:
            print("Error:", response.status_code, response.text)
    return total_calories

def generation_function(texts):
    _inputs = texts if isinstance(texts, list) else [texts]
    # Concatenate user-provided ingredients with common ingredients
    inputs = [prefix + ', '.join(COMMON_INGREDIENTS + inp.split(', ')) for inp in _inputs]
    inputs = tokenizer(
        inputs,
        max_length=256,
        padding="max_length",
        truncation=True,
        return_tensors="jax"
    )

    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        **generation_kwargs
    )
    generated = output_ids.sequences
    generated_recipe = target_postprocessing(
        tokenizer.batch_decode(generated, skip_special_tokens=False),
        special_tokens
    )

    return generated_recipe

In [13]:
def print_recipe(generated):
    for index, text in enumerate(generated):
        sections = text.split("\n")
        for section in sections:
            section = section.strip()
            if section.startswith("title:"):
                section = section.replace("title:", "")
                headline = "TITLE"
            elif section.startswith("ingredients:"):
                section = section.replace("ingredients:", "")
                headline = "INGREDIENTS"
            elif section.startswith("directions:"):
                section = section.replace("directions:", "")
                headline = "DIRECTIONS"

            if headline == "TITLE":
                print(f"[{headline}]: {section.strip().capitalize()}")
            elif headline == "INGREDIENTS":
                ingredients_list = section.split("--")
                ingredients_calories = estimate_calories([ing.strip() for ing in ingredients_list])
                section_info = [f"  - {i+1}: {info.strip().capitalize()}" for i, info in enumerate(section.split("--"))]
                print(f"[{headline}]:")
                print("\n".join(section_info))
                print(f"[Approximate Calories]: {ingredients_calories}")
            else:
                section_info = [f"  - {i+1}: {info.strip().capitalize()}" for i, info in enumerate(section.split("--"))]
                print(f"[{headline}]:")
                print("\n".join(section_info))

        print("-" * 130)

In [14]:
items = [
    "chicken breast, pasta, onion, tomato"
]

In [15]:
generated = generation_function(items)
print_recipe(generated)

[TITLE]: Chicken, noodle or garden salad
Calories in 1 tablespoon salt: 0.0
Calories in 1 teaspoon pepper: 5.7
Calories in 2 boneless skinless chicken breast: 391.8
Calories in 4 cups pasta, your choice: 773.5
Calories in 1 small onion, diced: 42.0
Calories in 1 tomato, diced: 22.4
[INGREDIENTS]:
  - 1: 1 tablespoon salt
  - 2: 1 teaspoon pepper
  - 3: 2 boneless skinless chicken breast
  - 4: 4 cups pasta, your choice
  - 5: 1 small onion, diced
  - 6: 1 tomato, diced
[Approximate Calories]: 1235.4
[DIRECTIONS]:
  - 1: Pre heat oven to 425 degrees.
  - 2: Combine salt and pepper in a small bowl and sprinkle evenly over the chicken breasts.
  - 3: Bake chicken until done, approx 20 minutes.
  - 4: Boil pasta and rinse with cold water.
  - 5: Let cool under cold running water.
  - 6: Combine pasta, chicken, and onion in a large bowl.
  - 7: Just before serving add tomato and toss together.
--------------------------------------------------------------------------------------------------